## Sessions - All Registrations

In [29]:
%load_ext autoreload
%autoreload
from oss_hugo.OSS_Schedule import OSS_Schedule
from oss_hugo.API_Hugo_OSS import API_Hugo_OSS

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows'    , 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [38]:
schedule = OSS_Schedule()
hugo     = API_Hugo_OSS()

df_sessions_participants = schedule.df_sessions_registered_participants()
df_sessions = hugo.df_sessions()
df_sessions_organizers  = df_sessions[['title','organizers']].dropna()
df_sessions_organizers['organizers_count'] = df_sessions_organizers['organizers'].apply(lambda x: len (x))

In [39]:
# merge both DataFrames
df_merge = df_sessions_organizers.merge(df_sessions_participants)
df_merge['total_count'] = df_merge['participants_count'] + df_merge['organizers_count']
df_merge = df_merge.sort_values(['participants_count','organizers_count'], ascending=False)

df_merge = df_merge.reset_index(drop=True)
df_merge[['title', 'total_count', 'participants_count','organizers_count','organizers','participants']]
;

''

In [40]:
# data= df_sessions_participants.set_index('title').to_dict(orient='index')
# pd.DataFrame(data)

# df_sessions              = hugo.df_sessions()
# df_sessions_organizers   = df_sessions[['title','organizers']].dropna()
# df_sessions_participants = schedule.df_sessions_registered_participants()
by_participants          = df_sessions_participants.set_index('title').to_dict(orient='index')
by_organizers            = df_sessions_organizers  .set_index('title').to_dict(orient='index')


In [41]:
set(by_participants)
set(by_organizers)
all_sessions = {}
for key, value in by_participants.items():
    participants_count = value.get('participants_count')
    all_sessions[key] = { 'title'       : key                ,
                          'participants': participants_count , 
                          'organizers'  : 0                  , 
                          'total'       : participants_count }

for key, value in by_organizers.items():
    organizers_count = value['organizers_count']
    if all_sessions.get(key):
        all_sessions[key]['organizers'] =  organizers_count
        all_sessions[key]['total'     ] += organizers_count
        pass
    else:
        all_sessions[key] = { 'title'        : key              ,
                              'participants' : 0                ,
                              'organizers'   : organizers_count , 
                              'total'        : organizers_count }
                                     

            
all_sessions = pd.DataFrame(all_sessions.values())
all_sessions = all_sessions[['title','total','organizers','participants']]
all_sessions = all_sessions.sort_values(['total'], ascending=False)
all_sessions

,title,total,organizers,participants
0,Securing the CI Pipeline,18,1,17
2,Creating Appsec metrics and visualisation,16,1,15
1,Agile Practices for Security Teams,16,1,15
4,State and future of threat modeling,13,1,12
5,Secrets Management,13,1,12
6,Integrating Security Tools in the SDL,13,1,12
8,Lightweight privacy threat modeling using LINDDUN,13,2,11
3,How to scale Threat Modeling,12,0,12
7,Application Security Verification Standard,12,1,11
9,Writing security tests to confirm vulnerabilities and fixes,12,1,11


In [27]:
len(all_sessions) # was 123

114

### test: check session's titles that don't exist

In [28]:
#this code prints the name of sessions that exist in participant's schedule but don't match a session title
schedule = OSS_Schedule()
df_sessions_participants = schedule.df_sessions_registered_participants()
titles = sorted(df_sessions_participants['title'].to_list())
sessions = hugo.sessions_oss()

for title in titles:
    if sessions.get(title) is None:
        print(title)
        
        
        

Create a Slack bot in Python
Describe different ways of implementing TM in agile organisations
How to scale Threat Modeling
SAMM - Model discussions
Share your Threat Models diagrams and create a Book
Threat Model Examples Project


In [191]:
#hugo.participants()

In [169]:
from oss_hugo.OSS_Participant import OSS_Participant

#hugo.qgrid_participants(['title','sessions'])
user = hugo.participants_oss()['Dinis Cruz']
user.fields()


['owasp_leadership',
 'title',
 'type',
 'image',
 'project_user',
 'project_contributor',
 'sessions',
 'website',
 'job_title',
 'company',
 'project_leader',
 'linkedin',
 'status',
 'aaa',
 'twitter']

In [174]:
user.field('sessions').append('aaa')

In [175]:
user.field('sessions')

['Using Wardley Maps on SOC',
 'Mapping boot camp',
 'Wardley Mapping - Coordinating functions within a PST organisation',
 'Wardley Mapping - Climatic Patterns and Using Doctrine',
 'Using Wardley Maps and Cynefin for Security',
 'Introduction to Cynefin Framework (Training Session)',
 'Using Cynefin Framework making strategic security decisions',
 'aaa']

In [177]:
user.save()

In [178]:
oss_bot = OSS_Participant('OSS_Bot')

In [179]:
oss_bot.exists()

False

In [180]:
oss_bot.create()

In [181]:
oss_bot.exists()

True

## Changing an user value

this will show how to change it

![image](https://user-images.githubusercontent.com/656739/58750905-abdba880-848f-11e9-9af5-06a9331bea41.png)




In [34]:
df_sessions_organizers[df_sessions_organizers.set_index('title').index.duplicated()]

,title,organizers,organizers_count
95,OWASP Juice Shop,[Bjoern Kimminich],1


In [35]:
df_sessions_organizers.set_index('title').to_dict(orient='index')

ValueError: DataFrame index must be unique for orient='index'.

True